In [ ]:
import requests
import pandas as pd
import datetime
from tqdm import tqdm

# API URL
url = "https://api.openweathermap.org/data/3.0/onecall/timemachine"

# Parameters for the API request
latitude = 7.252299424314035
longitude = 80.5940338059912
api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
units = "metric"

# Define the total number of API requests
total_requests = 888

# Create a progress bar using tqdm
progress_bar = tqdm(total=total_requests, unit="request")

# Create an empty DataFrame to store the weather data
weather_df = pd.DataFrame(columns=["Date", "Time", "Temperature", "Feels Like", "Pressure", "Humidity", "Dew Point", "Clouds", "Wind Speed", "Description"])

# Specify the start date and time
start_date = datetime.datetime(2020, 12, 31)
end_date = start_date + datetime.timedelta(hours=total_requests-1)  # 888 API requests

# Iterate over the hours from the start date to the end date
for dt in pd.date_range(start_date, end_date, freq="H"):
    # Calculate the UNIX timestamp for each hour
    dt_unix = int(dt.timestamp())

    # Construct the parameters for the API request
    params = {
        "lat": latitude,
        "lon": longitude,
        "dt": dt_unix,
        "appid": api_key,
        "units": units
    }

    # Send the API request
    response = requests.get(url, params=params)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Retrieve the response data
        data = response.json()

        # Extract the required information from the response
        weather_data = data["data"][0]
        date = dt.strftime("%Y-%m-%d")
        time = dt.strftime("%H:%M:%S")
        temperature = weather_data["temp"]
        feels_like = weather_data["feels_like"]
        pressure = weather_data["pressure"]
        humidity = weather_data["humidity"]
        dew_point = weather_data["dew_point"]
        clouds = weather_data["clouds"]
        wind_speed = weather_data["wind_speed"]
        description = weather_data["weather"][0]["description"]

        # Add the extracted data to the DataFrame
        weather_df = weather_df.append({"Date": date, "Time": time, "Temperature": temperature, 
                                        "Feels Like": feels_like, "Pressure": pressure, 
                                        "Humidity": humidity, "Dew Point": dew_point, 
                                        "Clouds": clouds, "Wind Speed": wind_speed, 
                                        "Description": description}, ignore_index=True)
        progress_bar.update(1)
    else:
        print("Error: Request failed.")

# Save the DataFrame to a CSV file
weather_df.to_csv('weather_data.csv', mode='a', header=False, index=False)
progress_bar.close()